# 将图片根据清晰度打分.数据保存到filtered_annotations.json

In [1]:
import os
import json
import cv2
import numpy as np
from tqdm import tqdm
from pathlib import Path

def calculate_laplacian_variance(image):
    """使用Laplacian梯度方法计算图像清晰度"""
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(image, cv2.CV_64F).var()

def calculate_tenengrad(image, ksize=3):
    """使用Tenengrad梯度方法计算图像清晰度"""
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=ksize)
    gy = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=ksize)
    gm = cv2.sqrt(gx**2 + gy**2)
    return gm.mean()

def calculate_brenner(image):
    """使用Brenner梯度方法计算图像清晰度"""
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    shift = 2
    brenner = np.abs(image[shift:] - image[:-shift])**2
    return brenner.mean()

def calculate_fft_energy(image):
    """使用FFT能量方法计算图像清晰度"""
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fft = np.fft.fft2(image)
    fft_shift = np.fft.fftshift(fft)
    magnitude_spectrum = 20 * np.log(np.abs(fft_shift) + 1)
    return np.mean(magnitude_spectrum)

def find_image_files(folder_path):
    """查找文件夹中的所有图片文件，返回文件名到路径的映射"""
    supported_formats = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif', '.JPG', '.JPEG', '.PNG', '.BMP', '.TIFF', '.TIF'}
    
    image_path_mapping = {}
    folder_path_obj = Path(folder_path)
    
    # 使用兼容的方式遍历所有文件
    for format_ext in supported_formats:
        pattern = f"*{format_ext}"
        try:
            # 使用 glob 而不是 rglob 避免路径拼接问题
            for img_path in folder_path_obj.glob(pattern):
                image_path_mapping[img_path.stem] = str(img_path)
            
            # 也检查子目录
            for img_path in folder_path_obj.rglob(pattern):
                image_path_mapping[img_path.stem] = str(img_path)
                
        except Exception as e:
            print(f"搜索 {format_ext} 文件时出错: {e}")
    
    return image_path_mapping

def add_clarity_scores_to_json(folder_path, json_file_path):
    """
    为JSON文件中的每个图片添加清晰度分数
    
    Args:
        folder_path: 图片文件夹路径
        json_file_path: JSON文件路径（直接修改原文件）
    """
    # 读取现有的JSON数据
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            existing_data = json.load(f)
        print(f"成功读取JSON文件: {json_file_path}")
    except Exception as e:
        print(f"读取JSON文件时出错: {e}")
        return
    
    # 查找所有图片文件
    image_path_mapping = find_image_files(folder_path)
    print(f"找到 {len(image_path_mapping)} 张图片")
    
    # 处理每个图像条目
    processed_count = 0
    not_found_count = 0
    
    for original_name, annotations in tqdm(existing_data.items(), desc="处理图片"):
        try:
            # 查找对应的图片文件
            image_path = image_path_mapping.get(original_name)
            if not image_path:
                # 尝试不同的名称变体
                possible_names = [
                    original_name,
                    original_name.replace('_', '-'),
                    original_name.replace('-', '_'),
                    original_name.lower(),
                    original_name.upper()
                ]
                
                for name in possible_names:
                    if name in image_path_mapping:
                        image_path = image_path_mapping[name]
                        break
                
                if not image_path:
                    print(f"未找到图片文件: {original_name}")
                    not_found_count += 1
                    continue
            
            # 读取图片
            img = cv2.imread(image_path)
            if img is None:
                print(f"无法读取图片: {image_path}")
                continue
            
            # 计算各种清晰度分数
            clarity_scores = {
                'laplacian_score': float(calculate_laplacian_variance(img)),
                'tenengrad_score': float(calculate_tenengrad(img)),
                'brenner_score': float(calculate_brenner(img)),
                'fft_energy_score': float(calculate_fft_energy(img))
            }
            
            # 更新每个标注项的清晰度分数
            for annotation in annotations:
                annotation.update(clarity_scores)
            
            processed_count += 1
            
        except Exception as e:
            print(f"处理图片 {original_name} 时出错: {e}")
    
    # 保存更新后的数据到原文件
    try:
        # 先创建备份
        backup_path = f"{json_file_path}.bak"
        if os.path.exists(json_file_path):
            import shutil
            shutil.copy2(json_file_path, backup_path)
            print(f"已备份原文件到: {backup_path}")
        
        # 保存新文件
        with open(json_file_path, 'w', encoding='utf-8') as f:
            json.dump(existing_data, f, indent=2, ensure_ascii=False)
        
        print(f"\n处理完成!")
        print(f"成功处理: {processed_count} 张图片")
        print(f"未找到图片: {not_found_count} 张")
        print(f"结果已保存到: {json_file_path}")
        
    except Exception as e:
        print(f"保存JSON文件时出错: {e}")

def main():
    # 配置参数
    path = Path("/workspace/models/SAHI/run_v7")
    chosen_dirs = [d / "raw_data" for d in path.iterdir() if d.is_dir()]
    print("子文件夹路径列表：", chosen_dirs)

    for folder_path in chosen_dirs:
        print(f"\n处理文件夹: {folder_path}")
        json_file_path = folder_path.parent / "raw_data_sliced_merge" / "filtered_annotations_v9.json"

        # 检查文件夹是否存在
        if not os.path.exists(folder_path):
            print("图片文件夹不存在!")
            continue
        
        # 检查JSON文件是否存在
        if not os.path.exists(json_file_path):
            print(f"JSON文件不存在: {json_file_path}")
            continue
        
        # 添加清晰度分数到JSON
        add_clarity_scores_to_json(folder_path, json_file_path)

if __name__ == "__main__":
    main()

子文件夹路径列表： [PosixPath('/workspace/models/SAHI/run_v7/ms1_0726-0809_11/raw_data'), PosixPath('/workspace/models/SAHI/run_v7/air1_0729-0813_5/raw_data'), PosixPath('/workspace/models/SAHI/run_v7/air1_0826-0909_21_no/raw_data'), PosixPath('/workspace/models/SAHI/run_v7/air2_0729-0813_04/raw_data'), PosixPath('/workspace/models/SAHI/run_v7/air2_0826-0909_25_no/raw_data'), PosixPath('/workspace/models/SAHI/run_v7/air2_0923-1007_02_wait/raw_data'), PosixPath('/workspace/models/SAHI/run_v7/jeff_0613-0624_04_ok/raw_data'), PosixPath('/workspace/models/SAHI/run_v7/jeff_0624-0702_01_ok/raw_data'), PosixPath('/workspace/models/SAHI/run_v7/jeff_0730-0813_01/raw_data'), PosixPath('/workspace/models/SAHI/run_v7/lloyd_0603-0618_31/raw_data'), PosixPath('/workspace/models/SAHI/run_v7/lloyd_0715-0729_04/raw_data'), PosixPath('/workspace/models/SAHI/run_v7/lloyd_0826-0909_04/raw_data'), PosixPath('/workspace/models/SAHI/run_v7/ms1_0605-0621_40/raw_data'), PosixPath('/workspace/models/SAHI/run_v7/ms1_0621

处理图片: 100%|██████████| 856/856 [12:47<00:00,  1.12it/s]


已备份原文件到: /workspace/models/SAHI/run_v7/ms1_0726-0809_11/raw_data_sliced_merge/filtered_annotations_v9.json.bak

处理完成!
成功处理: 856 张图片
未找到图片: 0 张
结果已保存到: /workspace/models/SAHI/run_v7/ms1_0726-0809_11/raw_data_sliced_merge/filtered_annotations_v9.json

处理文件夹: /workspace/models/SAHI/run_v7/air1_0729-0813_5/raw_data
成功读取JSON文件: /workspace/models/SAHI/run_v7/air1_0729-0813_5/raw_data_sliced_merge/filtered_annotations_v9.json
找到 149 张图片


处理图片: 100%|██████████| 59/59 [00:52<00:00,  1.12it/s]


已备份原文件到: /workspace/models/SAHI/run_v7/air1_0729-0813_5/raw_data_sliced_merge/filtered_annotations_v9.json.bak

处理完成!
成功处理: 59 张图片
未找到图片: 0 张
结果已保存到: /workspace/models/SAHI/run_v7/air1_0729-0813_5/raw_data_sliced_merge/filtered_annotations_v9.json

处理文件夹: /workspace/models/SAHI/run_v7/air1_0826-0909_21_no/raw_data
成功读取JSON文件: /workspace/models/SAHI/run_v7/air1_0826-0909_21_no/raw_data_sliced_merge/filtered_annotations_v9.json
找到 2991 张图片


...

已备份原文件到: /workspace/models/SAHI/run_v7/sw1_0808-0823_01/raw_data_sliced_merge/filtered_annotations_v9.json.bak

处理完成!
成功处理: 104 张图片
未找到图片: 0 张
结果已保存到: /workspace/models/SAHI/run_v7/sw1_0808-0823_01/raw_data_sliced_merge/filtered_annotations_v9.json

处理文件夹: /workspace/models/SAHI/run_v7/sw2_0725-0808_02/raw_data
成功读取JSON文件: /workspace/models/SAHI/run_v7/sw2_0725-0808_02/raw_data_sliced_merge/filtered_annotations_v9.json
找到 1479 张图片


处理图片: 100%|██████████| 318/318 [04:36<00:00,  1.15it/s]


已备份原文件到: /workspace/models/SAHI/run_v7/sw2_0725-0808_02/raw_data_sliced_merge/filtered_annotations_v9.json.bak

处理完成!
成功处理: 318 张图片
未找到图片: 0 张
结果已保存到: /workspace/models/SAHI/run_v7/sw2_0725-0808_02/raw_data_sliced_merge/filtered_annotations_v9.json

处理文件夹: /workspace/models/SAHI/run_v7/sw2_0808-0823_04/raw_data
成功读取JSON文件: /workspace/models/SAHI/run_v7/sw2_0808-0823_04/raw_data_sliced_merge/filtered_annotations_v9.json
找到 2133 张图片


处理图片: 100%|██████████| 895/895 [12:57<00:00,  1.15it/s]


已备份原文件到: /workspace/models/SAHI/run_v7/sw2_0808-0823_04/raw_data_sliced_merge/filtered_annotations_v9.json.bak

处理完成!
成功处理: 895 张图片
未找到图片: 0 张
结果已保存到: /workspace/models/SAHI/run_v7/sw2_0808-0823_04/raw_data_sliced_merge/filtered_annotations_v9.json
